In [40]:
import portfolio_advisor as pa
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
pa = pa.PortfolioAdvisor(risk_profile=3, current_date='2020-04-15')

Loaded: price_db_w.pkl
Price returns have been calculated.
Loaded: instruments_m.pkl
Loaded: simulatable_instruments.pkl.
Loaded: constraints.pkl.
Portfolio constraints have been set in matrice A and B such that Aw>=B for a risk profile number 3.


In [34]:
pa.optimize(drop_wt_threshold=0.005)

The risk free rate at this time is set to be 0.73%
Optimized weights have been estimated.
Weights < 0.005 are dropped to zero.


,weights,tracking_code,itemname
itemcode,,,
A069500,0.000000,K200,KODEX 200
A091160,0.000000,SEMI_KRX,KODEX 반도체
A091170,0.000000,BANK_KRX,KODEX 은행
A091180,0.000000,AUTO_KRX,KODEX 자동차
A102110,0.231641,K200,TIGER 200
A102780,0.000000,SAMSUNG_FN,KODEX 삼성그룹
A105190,0.000000,K200,KINDEX 200
A114260,0.100000,KTB_D3,KODEX 국고채3년
A122260,0.000000,BOK_D1,KOSEF 통안채1년


In [37]:
pa.w = pa.get_it_neat()

In [79]:
filepath = '../../data/processed/'
filename = 'investors_m'
investors_m = pd.read_pickle(filepath+filename+'.pkl')
investors_m = investors_m.rename(columns={'account_nm':'acc_no'}) # 임시조치. 컬럼명 acc_no가 적절

In [81]:
filepath = '../../data/processed/'
filename = 'balance_m'
balance_m = pd.read_pickle(filepath+filename+'.pkl')
balance_m['acc_no'] = 'acc1-01' # 임시 조치. acc_no 컬럼이 애초부터 컬럼으로 있어야 했음.

In [82]:
filename = 'balance_s'
balance_s = pd.read_pickle(filepath+filename+'.pkl')
balance_s['acc_no'] = 'acc1-01' # 임시 조치. acc_no 컬럼이 애초부터 컬럼으로 있어야 했음.

In [83]:
balance_m

,date,userid,asset_class,value,wt,acc_no
0,1/10/2020 1:00:00 PM,A01,현금성,700000.00,1.000000,acc1-01
1,1/10/2020 4:00:00 PM,A01,현금성,364555.00,0.518938,acc1-01
2,1/10/2020 4:00:00 PM,A01,주식,85527.70,0.121747,acc1-01
3,1/10/2020 4:00:00 PM,A01,채권,215383.10,0.306594,acc1-01
4,1/10/2020 4:00:00 PM,A01,대체,37036.80,0.052721,acc1-01
5,2/11/2020 9:00:00 AM,A01,현금성,373065.00,0.531376,acc1-01
6,2/11/2020 9:00:00 AM,A01,주식,96929.55,0.138062,acc1-01
7,2/11/2020 9:00:00 AM,A01,채권,158776.50,0.226154,acc1-01
8,2/11/2020 9:00:00 AM,A01,대체,73302.00,0.104408,acc1-01


In [64]:
balance_s[balance_s.투자원본=='Y']

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본,acc_no
0,1/10/2020 1:00:00 PM,A01,투자자1,현금성,C000001,현금,700000,1,700000,1.0,700000.0,1.0,20200110 13:00현금성,Y,800-01


In [65]:
from user import User

In [70]:
users = load_users(userinfo=investors_m, balance=balance_s)

NameError: name 'load' is not defined

In [ ]:
def load_users(userinfo, book, reference_date='signup'):
    r"""
    users에서 읽어온 userid로 사용자 클래스(User)의 인스턴스를 만든다.
    각 User 인스턴스는 딕셔너리인 users에 {userid:User instance} 형식으로 저장되고,
    이 딕셔너리 users가 반환된다.
    
    현재 가정
    -------
    가입되어 있는 사용자만 로드할 수 있다. 충분한 사용자가 가입되어 있다.
    사용자마다 포트폴리오 운용을 위한 충분한 잔고가 있다.
    
    Parameters
    ----------
    userinfo : DataFrame
        사용자 정보 테이블. investors_m 형식이다.
        investors_m 형식의 pandas DataFrame 인스턴스.
    book : DataFrame
        원장 테이블. 계좌 내 보유종목 등 상세내역이 포함되어 있다. balance_s 형식이다.
        이 원장 내 잔고를 근거로 RA가 포트폴리오를 추천하고, 매매를 위한 주문지를 제시한다.
    reference_date : 'signup' or 'latest'.
        이 값이 'signup'이면 가입했던 시점의 잔고를 기준으로 포트폴리오 추천과 매매 주문지 작성 작업이 실행된다.
        'latest'이면 최근 날짜 기준의 잔고를 사용한다.
        Default is 'signup'.

    Return
    ------
    users : dict
        userid를 key값으로 하고, User 클래스의 인스턴스를 value로 하는 파이썬 딕셔너리임.
    """
    users = {}
    
    for userid in userinfo.userid.unique():
        users[userid] = User(username=)

In [91]:
investors_m[investors_m.userid=='A01']

,userid,name,acc_no,profile_code
0,A01,투자자1,acc1-01,2
1,A01,투자자1,acc1-02,2


In [93]:
investors_m[investors_m.userid=='A01'].name

0    투자자1
1    투자자1
Name: name, dtype: object

In [ ]:
def create_user(userid,
                username,
                risk_profile,
                deposit=700000,
                access_path='../../processed/',
                access_name='investors_m.pkl'):
    r"""
    이용자 1명을 새로 만들거나(계좌도 1개 생성된다), 기존 이용자에게 계좌 1개를 새로 발급한다.
    
    Parameters
    ----------
    userid : str
        사용자 아이디
    username : str
        사용자 이름
    risk_profile : int
        투자자 성형 분류번호
    deposit : float
        입금액. Default is 700,000.
    access_path: str
        현재까지의 이용자 정보가 저장된 파일 또는 DB의 경로
    access_name: str
        현재까지의 이용자 정보가 저장된 파일명 또는 DB 테이블명
        
    Return
    ------
    user : User
        주어진 정보로 User 클래스의 인스턴스를 만들어 반환한다.
    """
    
    userinfo = pd.read_pickle(access_path+access_name)
    
    user = User(userid=userid, username=username)

In [96]:
investors_m.iloc[0].userid

'A01'